In [1]:
# Importation des librairies
import pandas as pd

pd.set_option('display.max_colwidth', -1)

import sys
import os
import time

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), ".."))

from utils_project import load_saved_file, read_feel, process_label, process_nlp, vectorize, apply_vectorization

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
PATH_DATA = "/home/erwan/Centrale/OSY/Projet_OSY/sentiment-analysis/data/kaggle-traduction/translate_final.csv"

TYPE_VECTOR = "tf-idf"
UP_SAMPLING = True

In [3]:
data = pd.read_csv(PATH_DATA, sep=";", encoding='utf-8')
data= data.drop(['Unnamed: 0', 'content'], axis=1)
data = data.fillna("")
data.head()

,sentiment,translation,translation_yandex,translation_microsoft,translation_google
0,empty,"@ tiffanylue je sais, j’écoutais la mauvaise habitude avant et j’ai commencé à flipper à cause de ça = [",@tiffanylue je sais je suis listenin à mauvaise habitude plus tôt et j’ai commencé freakin à sa part = [,@tiffanylue je sais je suis listenin à mauvaise habitude plus tôt et j’ai commencé freakin à sa part = [,@ tiffanylue je sais que j’écoutais la mauvaise habitude plus tôt et j’ai commencé freakin de sa part = [
1,sadness,Allongé sur mon lit avec un mal de tête ughhhh ... j’attend ton appel ...,Laporal n lit avec un mal de tête pouah... waitin sur votre appel...,Laporal n lit avec un mal de tête Pouah... waitin sur votre appel...,Layin n lit avec un mal de tête ughhhh ... attendez votre appel ...
2,sadness,Cérémonie funéraire... vendredi sombre...,Cérémonie funéraire... vendredi sombre...,Cérémonie funéraire... vendredi sombre...,Cérémonie funèbre ... triste vendredi ...
3,enthusiasm,veut sortir avec des amis BIENTOT!,veut passer du temps avec des amis bientôt !,veut passer du temps avec des amis bientôt !,veut sortir avec des amis bientôt!
4,neutral,"@dannycastillo On veut échanger avec quelqu’un qui a des billets pour Houston, mais personne ne le fera.","@dannycastillo nous voulons commerce avec quelqu'un qui a des billets de Houston, mais personne ne va.","@dannycastillo nous voulons commerce avec quelqu'un qui a des billets de Houston, mais personne ne va.","@dannycastillo Nous voulons échanger avec quelqu&#39;un qui a des billets pour Houston, mais personne ne le fera."


In [4]:
# On ne garde que certains sentiments
data.loc[data.sentiment == 'anger', ['sentiment']] = 'hate'
data.loc[data.sentiment == 'enthusiasm', ['sentiment']] = 'happiness'
data.loc[data.sentiment == 'fun', ['sentiment']] = 'happiness'
data = data.loc[~data.sentiment.isin(['boredom', 'relief', 'empty'])]
data.sentiment.value_counts()

worry        7433
neutral      6340
sadness      4828
happiness    4596
love         2068
surprise     1613
hate         1285
Name: sentiment, dtype: int64

In [5]:
data.translation_google = data.translation_google.map(lambda sent : sent.replace("&#39;", "'"))
data.head()

,sentiment,translation,translation_yandex,translation_microsoft,translation_google
1,sadness,Allongé sur mon lit avec un mal de tête ughhhh ... j’attend ton appel ...,Laporal n lit avec un mal de tête pouah... waitin sur votre appel...,Laporal n lit avec un mal de tête Pouah... waitin sur votre appel...,Layin n lit avec un mal de tête ughhhh ... attendez votre appel ...
2,sadness,Cérémonie funéraire... vendredi sombre...,Cérémonie funéraire... vendredi sombre...,Cérémonie funéraire... vendredi sombre...,Cérémonie funèbre ... triste vendredi ...
3,happiness,veut sortir avec des amis BIENTOT!,veut passer du temps avec des amis bientôt !,veut passer du temps avec des amis bientôt !,veut sortir avec des amis bientôt!
4,neutral,"@dannycastillo On veut échanger avec quelqu’un qui a des billets pour Houston, mais personne ne le fera.","@dannycastillo nous voulons commerce avec quelqu'un qui a des billets de Houston, mais personne ne va.","@dannycastillo nous voulons commerce avec quelqu'un qui a des billets de Houston, mais personne ne va.","@dannycastillo Nous voulons échanger avec quelqu'un qui a des billets pour Houston, mais personne ne le fera."
5,worry,Re-ping @ ghostridah14: pourquoi n’êtes-vous pas allé au bal de promo? Parce que ma BF n'a pas aimé mes amis,@ghostridah14 re-ping : Pourquoi n’allez-vous pas au bal ? BC mon bf n’aimait pas mes amis,@ghostridah14 re-ping : Pourquoi n’allez-vous pas au bal ? BC mon bf n’aimait pas mes amis,Re-ping @ ghostridah14: pourquoi n’êtes-vous pas allé au bal de promo? BC mon petit ami n'a pas aimé mes amis


In [18]:
data.loc[data.translation_google.str.contains("&")]

,sentiment,translation,translation_yandex,translation_microsoft,translation_google
41,fun,@ DavidArchie &lt;3 tu vas être le meilleur twitter;) car tes incroyable lol. Viens au canada ferais n’importe quoi pour te voir jouer,@DavidArchie &lt; 3 tu vas être le premier twitter ;) causer votre incroyable lol. arrivé au canada ferait n’importe quoi pour voir que vous effectuez,@DavidArchie &lt; 3 tu vas être le premier twitter ;) causer votre incroyable lol. arrivé au canada ferait n’importe quoi pour voir que vous effectuez,@ DavidArchie &lt;3 tu vas être le premier twitter;) cause ton incroyable lol. venir au canada ferait n'importe quoi pour te voir jouer
69,happiness,,"Génial de voir Oin &amp; Cynthia. Si heureux. Le dîner était excellent, mignon petit endroit. Tant pis Oin est tombé malade par la suite.","Génial de voir Oin &amp; Cynthia. Si heureux. Le dîner était excellent, mignon petit endroit. Tant pis Oin est tombé malade par la suite.","Tellement génial de voir Oin &amp; Cynthia. Tellement heureux. Le dîner était super, petit endroit mignon. Dommage que Oin soit tombé malade par la suite."
119,sadness,,Allergies sucent noix de canards. &lt; ===8@8=== &gt;,Allergies sucent noix de canards. &lt; ===8@8=== &gt;,Les allergies sucent les noix de canards. &lt;===== 8 @ 8 =====&gt;
171,sadness,,"@heresmyhello92 nous détestons changer, alors bien sûr. Nous sommes des fans pour la vie. &lt; 3 Oh et si nous n’avons pas rencontré Rob par alors quelque chose a terriblement mal tourné.","@heresmyhello92 nous détestons changer, alors bien sûr. Nous sommes des fans pour la vie. &lt; 3 Oh et si nous n’avons pas rencontré Rob par alors quelque chose a terriblement mal tourné.","@ heresmyhello92 Nous détestons le changement, alors bien sûr. Nous sommes des fans pour la vie. &lt;3 Oh, et si nous n'avons pas rencontré Rob d'ici là, quelque chose a mal tourné."
187,worry,,A passé la nuit dernière dans A &amp; E (ER). Femme emmêlé les roues avec la fille et a frappé le pont. Luxation et fracture a abouti.,A passé la nuit dernière dans A &amp; E (ER). Femme emmêlé les roues avec la fille et a frappé le pont. Luxation et fracture a abouti.,J'ai passé la nuit à A &amp; E (ER). Femme enchevêtrée roues avec fille et frapper le pont. Il en a résulté une luxation et une fracture.
224,neutral,,sur @lizzziemonster &amp; @jamiejamess je suis tellement faim,sur @lizzziemonster &amp; @jamiejamess je suis tellement faim,avec @lizzziemonster &amp; @jamiejamess je suis soooo faim
230,worry,,"@__sugar oh non, je suis toujours ici ;) &lt; 3","@__sugar oh non, je suis toujours ici ;) &lt; 3",@__sugar oh non je suis toujours là;) &lt;3
255,surprise,,"@Gurlstrange lieu de l’épicentre Wheelock, 250 unités &amp; pour touristes seulement. A partir d’aujourd'hui","@Gurlstrange lieu de l’épicentre Wheelock, 250 unités &amp; pour touristes seulement. A partir d’aujourd'hui","@Gurlstrange epicenter Wheelock Place, 250 unités &amp; réservé aux touristes. À partir d'aujourd'hui"
291,sadness,,@xoshayzers j’ai knoww choses ne seront pas la samee &lt; / 3,@xoshayzers j’ai knoww choses ne seront pas la samee &lt; / 3,@xoshayzers je sais que les choses ne seront pas les mêmes &lt;/ 3
307,neutral,,"@jhicks vous êtes diabétique ? donc mon père. &amp; je pourrais être trop, pas sûr. êtes-vous toujours autorisés à manger sucre toute la journée ?","@jhicks vous êtes diabétique ? donc mon père. &amp; je pourrais être trop, pas sûr. êtes-vous toujours autorisés à manger sucre toute la journée ?","@ jhicks vous êtes diabétique? tout comme mon père. &amp; Je pourrais être aussi, pas sûr. êtes-vous toujours autorisé à manger du sucre toute la journée?"


In [6]:
# Traitement NLP des données
t1 = time.time()
data['nlp_yandex'] = process_nlp(data['translation_yandex'], feel=False, process_type='stem')
t2 = time.time()
print('NLP processed, duration {}s'.format(t2-t1))

NLP processed, duration 21.36907386779785s


In [15]:
# Séparation features / label
x_train, y_train = data.nlp_yandex, data.sentiment

In [19]:
# Vectorisation des données
t5 = time.time()
feat_train, vectorizer = vectorize(x_train, type_vector=TYPE_VECTOR)
t6 = time.time()
print("Text vectorized, duration : {}s".format(round(t6-t5, 4)))

Text vectorized, duration : 0.6781s


In [22]:
# Up-sampling
if UP_SAMPLING:
    from imblearn.over_sampling import RandomOverSampler
    t7 = time.time()
    upsamp = RandomOverSampler(random_state=777)
    #upsamp = SMOTE(random_state=777, k_neighbors=3)
    
    feat_train, y_train = upsamp.fit_sample(feat_train, y_train)
    t8 = time.time()
    
    print(pd.DataFrame({'emotion' : y_train}).emotion.value_counts())
    print("Up-sampling done, duration : {}s".format(t8-t7))

hate         7433
neutral      7433
worry        7433
love         7433
sadness      7433
happiness    7433
surprise     7433
Name: emotion, dtype: int64
Up-sampling done, duration : 0.0659799575805664s


In [31]:
# cross validation
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

kf = KFold(n_splits=5, shuffle=True, random_state=123)
log_reg = LogisticRegression(n_jobs=-1, multi_class='auto')
rand_for = RandomForestClassifier(n_jobs=-1, n_estimators=200)

score_log = cross_val_score(rand_for, feat_train, y_train, n_jobs=-1, cv=kf).mean()

print('score_log : {}'.format(score_log))

score_log : 0.7621226553979167
